In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

NEXT: Will add something to get all of the players from the match, and then count the defensive players to determine back 4 or back 5

Code will be -- take all of the elements where the class "player-position" is DC... get a count, if it's two then the team will be in a back four, if there are 3 then it's a back five with wingbacks

In [2]:
# d = range(18202,18962)
match_range = range(18202,18400)

In [3]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [4]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     positions = soup.find_all("span", {"class": "player-position"})

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [5]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [6]:
all_res_data = backup_all_res_data

In [7]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [8]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [9]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [10]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [11]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
situation_list = []
result_list = []
team_list = []
opposing_team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    situation = []
    result = []
    team = []
    opposing_team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'a_team':
                opposing_team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'h_team':
                opposing_team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    opposing_team_list.append(opposing_team)
    xG_list.append(xG)
    situation_list.append(situation)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [12]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
opposing_team_list = [val for sublist in opposing_team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
situation_list = [val for sublist in situation_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]
#Flattening nested lists

In [13]:
col_names = ['match_id','x','y','xG','minute','situation','result','team','opposing_team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,situation_list,result_list,team_list,opposing_team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [14]:
df.isna().sum()
#Checking for null values

index            0
match_id         0
x                0
y                0
xG               0
minute           0
situation        0
result           0
team             0
opposing_team    0
h_a              0
player           0
home_score       0
away_score       0
dtype: int64

# xG

In [15]:
# test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [16]:
df['xG'] = df['xG'].astype(float)
df['match_id'] = df['match_id'].astype(int)

In [17]:
xg_total = df.groupby('team')['xG'].sum()
#Count total xG for every team

In [18]:
xg_total = pd.DataFrame(data=xg_total)
# xg_total = xg_total.reset_index(level=0)

In [19]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [20]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [21]:
xg_total['matches_played'] = df.groupby('team')['match_id'].nunique()
#Get number of matches played for each team

In [22]:
xg_total = xg_total.reset_index(level=0)

In [23]:
xg_total['avg_xG_diff'] = round(xg_total['xG_difference'] / xg_total['matches_played'], 2)
xg_total['avg_xG_for'] = round(xg_total['xG'] / xg_total['matches_played'], 2)
xg_total['avg_xGA'] = round(xg_total['xGA'] / xg_total['matches_played'], 2)

In [24]:
xg_total.isna().sum()

team              0
xG                0
xGA               0
xG_difference     0
matches_played    0
avg_xG_diff       0
avg_xG_for        0
avg_xGA           0
dtype: int64

In [25]:
xg_total.to_excel("prem_xG_data.xlsx")

# Non Pen xG

In [26]:
non_pen_xg = df.groupby('team')
non_pen_xg = non_pen_xg.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())
#Getting non pen xG and adding to df
#Not working because this groupby result does not have a name so can't go into a df

In [27]:
non_pen_xg = non_pen_xg.sort_values(ascending=False)
#sort so that teams match up

In [ ]:
non_pen_xga = 

In [ ]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [28]:
xg_total['non_pen'] = ''

In [29]:
xg_total['non_pen_xg'] = g.values
#Now need to match for team

In [30]:
xg_total

,team,xG,xGA,xG_difference,matches_played,avg_xG_diff,avg_xG_for,avg_xGA,non_pen,non_pen_xg
0,Arsenal,16.932950,7.413862,9.519088,8,1.19,2.12,0.93,,19.887952
1,Aston Villa,8.194168,10.962579,-2.768411,8,-0.35,1.02,1.37,,16.932950
2,Bournemouth,4.111060,14.348897,-10.237837,8,-1.28,0.51,1.79,,16.387827
3,Brentford,12.399633,10.729053,1.670581,8,0.21,1.55,1.34,,15.854558
4,Brighton,13.656979,7.915632,5.741346,7,0.82,1.95,1.13,,12.861284
5,Chelsea,9.727666,10.314740,-0.587075,7,-0.08,1.39,1.47,,11.638464
6,Crystal Palace,8.752718,12.852809,-4.100092,7,-0.59,1.25,1.84,,11.373472
7,Everton,10.090060,13.131903,-3.041843,8,-0.38,1.26,1.64,,10.612038
8,Fulham,9.843347,18.857684,-9.014337,8,-1.13,1.23,2.36,,10.090060
9,Leeds,9.857185,10.728392,-0.871207,7,-0.12,1.41,1.53,,9.857185


In [31]:
xg_total = xg_total.sort_values(by='avg_xG_diff', ascending=False).reset_index(drop=True)

Extra Code

In [32]:
# for i in range(len(df)):
#     df['home_current_score'][i+1] = new_home[i]
#     df['away_current_score'][i+1] = new_away[i]
    
#Setting the value one after the goal scored as equal to one, because technically at the time of the goalscoring shot, the 
#score would not have changed yet -- so we want to change the one after

In [33]:
df['away_score'] = (df['away_goal_scored'] == 1).cumsum()
df['home_score'] = (df['home_goal_scored'] == 1).cumsum()
#This sums for every time a goal is scored in the entire df -- need to do it for each individual match_id

KeyError: 'away_goal_scored'